In [5]:
import numpy as np
import os

In [32]:
# 文件的根目录
root = r'D:\md_data\AI_potential'

In [26]:
file_name = '3000k-md.out'
# 文件路径
file_path = os.path.join(root,file_name)

In [27]:
read_file =  file_path
#
atoms_list_alat = []
num_atoms = None

with open(read_file, "r") as f:
    lines = f.readlines()

i = 0
while i < len(lines):
    line = lines[i].strip()

    # Step 1: 获取原子数
    if num_atoms is None and line.startswith("number of atoms/cell"):
        parts = line.split("=")
        if len(parts) == 2:
            num_atoms = int(parts[1].strip())

    # Step 2: 遇到新的坐标段
    elif "new positions in cart coord (alat unit)" in line:
        if num_atoms is None:
            raise ValueError("num_atoms 未定义，可能没有正确读取原子数量。")

        temp_list = []
        for j in range(i + 1, i + 1 + num_atoms):
            if j >= len(lines):
                break
            atom_line = lines[j].strip()
            tokens = atom_line.split()
            if len(tokens) >= 4:
                element = tokens[0]
                coords = list(map(float, tokens[1:4]))
                temp_list.append([element] + coords)

        atoms_list_alat.append(temp_list)
        i += num_atoms  # 跳过这 num_atoms 行

    i += 1

# 输出确认
print(f"共找到 {len(atoms_list_alat)} 个坐标段，每段 {num_atoms} 个原子。")
print("第一组前3个原子示例：")
for atom in atoms_list_alat[0][:3]:
    print(atom)


共找到 47 个坐标段，每段 192 个原子。
第一组前3个原子示例：
['O', 0.24505462, -0.000554297, -0.002587389]
['O', 0.625181806, 0.215201148, 0.002736926]
['O', 0.12752303, 0.212948103, -0.003096283]


In [28]:
alat = 19.94530335  # 单位：Bohr
bohr_to_angstrom = 0.529177

# 遍历 atoms_list_alat 中的所有帧和原子，转换坐标
atoms_list_angstrom = []

for frame in atoms_list_alat:
    frame_converted = []
    for atom in frame:
        element = atom[0]
        coords = [c * alat * bohr_to_angstrom for c in atom[1:4]]
        frame_converted.append([element] + coords)
    atoms_list_angstrom.append(frame_converted)
#
print("第一组前3个原子示例：")
for atom in atoms_list_angstrom[0][:3]:
    print(atom)

第一组前3个原子示例：
['O', 2.586452460778619, -0.005850380783076875, -0.02730884504867335]
['O', 6.598541258119193, 2.2713611308653707, 0.028887147639448635]
['O', 1.3459540356735393, 2.247581151591791, -0.03268001551905858]


# 输出xyz文件

In [29]:
save_name = 'AIMD_3000k.xyz'
# 
save_path = os.path.join(root,save_name)

In [30]:
def save_atoms_to_file(atoms_list_angstrom, save_path):
    with open(save_path, "w") as f:
        for frame in atoms_list_angstrom:
            num_atoms = len(frame)
            f.write(f"{num_atoms}\n")
            f.write("Atoms\n")
            for atom in frame:
                element = atom[0]
                x, y, z = atom[1:]
                f.write(f"{element:<4s} {x:12.6f} {y:12.6f} {z:12.6f}\n")

In [31]:
# 使用存储函数
save_atoms_to_file(atoms_list_angstrom, save_path)